# SEC 10-K/10-Q Analysis - RAPTOR RAG Project Plan

## Project Overview
AI-powered system for analyzing complete SEC 10-K and 10-Q filings (1993-2024) using RAPTOR RAG (Recursive Adaptive Processing and Topical Organizational Retrieval). The system will create an enhanced knowledge base from financial filings that users can query interactively to identify year-over-year changes, patterns, and potential anomalies.

**Data Coverage:** 1993-2024 (31 years of SEC EDGAR filings, ~51 GB)

---

## Core Architecture

### Infrastructure
- **Deployment**: AWS EC2 instance with GPU (in progress)
- **Model Hosting**: Ollama for local LLM deployment
- **User Interface**: Open WebUI for interactive queries
- **Data Storage**: Cloud-based storage for processed embeddings and knowledge base

### Architecture Diagram

![System Architecture](diagrams/architecture.png)

### RAPTOR RAG System
Unlike traditional RAG systems that use simple similarity search, RAPTOR implements:
- **Hierarchical Clustering**: Multi-level organization (global + local) using UMAP and Gaussian Mixture Models
- **Recursive Summarization**: 3-level hierarchical summaries capturing both granular details and high-level concepts
- **Enhanced Context Retrieval**: Cluster-aware retrieval providing richer context for LLM queries

---

## Technical Stack

### NLP & ML
- **LLM Model**: llama3-sec (Arcee AI, Llama-3-70B based, fine-tuned for SEC filings)
  - Source: `arcee-ai/llama3-sec` on Ollama
  - Trained on 72B tokens of SEC filing data (20B checkpoint available)
  - Deployment: Via Ollama (`ollama pull arcee-ai/llama3-sec`)
  - Fallback models: gpt-oss (13 GB), qwen2.5:1.5b (986 MB)
- **RAPTOR Implementation**: Adapted from FinGPT's `FinancialReportAnalysis/utils/rag.py`
  - Source: https://github.com/AI4Finance-Foundation/FinGPT
  - Custom implementation in `src/models/raptor.py`
- **Embeddings**: Sentence Transformers (`all-MiniLM-L6-v2`) for local, cost-free embedding generation
- **Clustering**: UMAP (dimensionality reduction) + scikit-learn GMM
- **LLM Interface**: Ollama (primary) or OpenAI API (testing/comparison)

### Data Processing
- **Chunking**: LangChain `RecursiveCharacterTextSplitter` (~2000 tokens/chunk)
- **Vector Storage**: ChromaDB for efficient retrieval
- **Data Format**: JSON/Parquet for structured storage

### Libraries
- `langchain`, `langchain_community` - LLM orchestration
- `sentence-transformers` - Local embeddings
- `umap-learn` - Dimensionality reduction
- `scikit-learn` - Clustering algorithms (GMM)
- `pandas`, `numpy` - Data manipulation
- `requests` - SEC EDGAR API access
- `ollama` - Python client for Ollama

---

## Data Scope

### Current Data Holdings
- **Time Period:** 1993-2024 (31 years)
- **Data Size:** ~51 GB
- **Data Location:** `data/external/`
- **Filing Types:** Complete 10-K (annual reports) and 10-Q (quarterly filings)
- **Processing Scope:** Full filing text (all sections)
- **Analysis Focus:** Year-over-year changes, topic trends, boilerplate vs. substantive disclosure

**Why Process Complete Filings:**
- RAG enables users to ask questions about ANY section (not just risk factors)
- RAPTOR clustering naturally organizes content by topic regardless of section
- Maximizes system flexibility and future-proofs the knowledge base
- Supports queries like: "How did revenue recognition policies change?" or "Compare executive compensation across years"

---

## RAPTOR Pipeline Flowchart

![RAPTOR Pipeline](diagrams/raptor_pipeline.png)

---

## Data Processing Workflow

![Data Processing Workflow](diagrams/data_processing_workflow.png)

---

## Implementation Phases

### Phase 1: Model Research & Setup (Week 1) ✅ COMPLETE
**Objectives:**
- [x] Clarify FinGPT components: FinGPT-v3 (LLM model) vs RAPTOR (Python implementation)
- [x] Set up Ollama and test local LLM deployment
- [x] Evaluate and download appropriate models for SEC filing analysis
- [x] Set up project structure (`src/`, `data/`, `notebooks/`, `dashboard/`)
- [ ] Copy and adapt RAPTOR class from FinGPT's `rag.py` (deferred to Phase 3)
- [ ] Create base `Raptor` class skeleton in `src/models/raptor.py` (deferred to Phase 3)

**Completed Actions:**
- ✅ Installed Ollama v0.12.5 on Windows
- ✅ Downloaded qwen2.5:1.5b (986 MB) - lightweight model for testing
- ✅ Downloaded gpt-oss (13 GB) - reasoning-capable model
- ✅ Downloading llama3-sec (50 GB) - SEC-specific model by Arcee AI (in progress)
- ✅ Verified Python ollama package integration
- ✅ Tested model inference via command line and Python

**Model Selection for Project:**
Primary model: **llama3-sec** (arcee-ai/llama3-sec)
- Trained on 72B tokens of SEC filing data (currently at 20B checkpoint)
- Based on Llama-3-70B architecture
- Specialized for SEC data analysis, investment analysis, risk assessment
- 50 GB download, 4-bit quantized for ~35-40 GB RAM usage
- Status: Currently downloading

Fallback models:
- **gpt-oss** (13 GB) - General purpose with reasoning capabilities
- **qwen2.5:1.5b** (986 MB) - Lightweight for quick testing

**Key Clarification:**
- ✅ **FinGPT-v3** = Fine-tuned LLM model (downloadable from Hugging Face, runnable in Ollama)
- ✅ **RAPTOR** = Python implementation for hierarchical clustering/summarization (we copy the code)
- ✅ **fingpt-rag** = Deprecated project name (not a model), replaced by newer implementations
- ✅ **llama3-sec** = Domain-specific SEC filing model (best fit for this project)

---

### Phase 2: Data Processing Pipeline (Week 2) - IN PROGRESS (Sample Testing)
**Objectives:**
- [ ] Extract filings from downloaded archives (1993-2024, ~51 GB)
- [ ] Parse complete 10-K/10-Q text from HTML/XML/SGML formats
- [ ] Implement document chunking (2000 token chunks with tiktoken)
- [ ] Generate embeddings using local Sentence Transformers
- [ ] Store structured data (chunks + metadata) in JSON/Parquet

**Key Files:**
- `src/data/filing_extractor.py` - Unzip archives, extract full filing text
- `src/data/text_processor.py` - Clean text, chunk into 2000-token segments
- `src/models/embedding_generator.py` - Embedding creation

**Current Status:**
- Working with sample filings to validate processing approach
- Full pipeline implementation deferred until Phase 1 model setup is complete

**Validation:**
- Test on 3-5 sample filings from different time periods (1995, 2010, 2024)
- Verify text extraction accuracy across HTML/XML/SGML formats
- Confirm chunking preserves semantic coherence

---

### Phase 3: RAPTOR System Implementation (Week 3)
**Objectives:**
- [ ] Implement hierarchical clustering (adapted from FinGPT's RAPTOR):
  - Global clustering (UMAP → GMM with BIC for optimal cluster count)
  - Local clustering (secondary refinement within global clusters)
- [ ] Build recursive summarization engine (3 levels deep)
- [ ] Create enhanced knowledge base combining:
  - Original document chunks
  - Level 1 summaries (cluster summaries)
  - Level 2 summaries (summary of summaries)
  - Level 3 summaries (highest abstraction)
- [ ] Implement cluster-aware retrieval mechanism

**Key Methods in `Raptor` class (adapted from FinGPT):**
```python
def global_cluster_embeddings(embeddings, dim, n_neighbors, metric="cosine")
def local_cluster_embeddings(embeddings, dim, num_neighbors=10)
def get_optimal_clusters(embeddings, max_clusters=50)
def GMM_cluster(embeddings, threshold, random_state=0)
def perform_clustering(embeddings, dim, threshold)
def recursive_embed_cluster_summarize(texts, level=1, n_levels=3)
```

**Source Reference:**
- Original implementation: https://github.com/AI4Finance-Foundation/FinGPT/blob/master/fingpt/FinGPT_FinancialReportAnalysis/utils/rag.py

**Testing:**
- Validate clustering quality on sample documents
- Review generated summaries for coherence
- Ensure topics are properly grouped (e.g., all revenue-related content clusters together)

---

### Phase 4: LLM Integration & Deployment (Week 4)
**Objectives:**
- [ ] Set up Ollama on EC2 instance with llama3-sec model
- [ ] Deploy Open WebUI for user interaction
- [ ] Integrate RAPTOR knowledge base with LLM query system
- [ ] Implement query handling for diverse topics across filings
- [ ] Create sample query templates for common use cases

**Integration Workflow:**
1. User submits query via Open WebUI
2. RAPTOR retrieves relevant chunks + hierarchical summaries
3. Context passed to Ollama LLM (llama3-sec)
4. LLM generates response with supporting evidence
5. Results displayed in WebUI

**Example Queries:**
- "What cyber risks did Apple disclose in 2023?"
- "How have revenue recognition policies evolved from 2010 to 2024?"
- "Compare executive compensation disclosures between tech companies"
- "Show boilerplate vs. substantive language in risk disclosures"

**Deliverables:**
- Functional Open WebUI interface
- End-to-end query processing pipeline
- Documentation for common queries

---

## RAPTOR vs. Traditional RAG Comparison

| Feature | Traditional RAG | RAPTOR RAG |
|---------|----------------|------------|
| Text Processing | Simple chunking | Recursive, hierarchical |
| Clustering | None or basic | Multi-level (global + local) |
| Summarization | None or single-level | Recursive, 3-level |
| Context Selection | Similarity-based only | Cluster-aware + similarity |
| Document Understanding | Flat representation | Hierarchical representation |
| Knowledge Integration | Direct chunks only | Chunks + multi-level summaries |

**Why RAPTOR for Financial Filings?**
- Financial documents have hierarchical structure (sections, subsections, themes)
- YoY analysis requires understanding both granular changes and high-level shifts
- Boilerplate detection benefits from cluster analysis (repetitive language clusters together)
- Complex queries need multi-level context (e.g., "How did regulatory disclosures evolve?")
- Historical coverage (1993-2024) enables long-term trend analysis

---

## Success Metrics
- [ ] Successfully process 90%+ of downloaded filings (1993-2024) into knowledge base
- [ ] Clustering produces coherent, interpretable topic groups
- [ ] Generated summaries accurately capture content at each hierarchical level
- [ ] LLM queries return relevant, accurate responses with supporting evidence
- [ ] System responds to queries in <10 seconds (including retrieval + generation)
- [ ] Manual validation: Test 10 diverse queries across different topics and decades, verify accuracy

---

## Key Advantages of AI-First Approach
1. **No Manual Feature Engineering**: LLM infers patterns from enhanced context (vs. building YoY diff algorithms)
2. **Flexible Queries**: Users can ask arbitrary questions about ANY topic or section
3. **Semantic Understanding**: Detects substantive changes even when wording differs
4. **Scalable**: Adding new filings just requires re-running RAPTOR pipeline
5. **Explainable**: LLM can cite specific sections supporting its conclusions
6. **Historical Depth**: 31 years of data enables long-term trend analysis

---

## Technical Challenges & Mitigations

### Challenge 1: Embedding Generation at Scale
- **Issue**: Processing 31 years of complete filings (~51 GB) requires significant compute power
- **Solution**: Use EC2 GPU instance, batch processing, cache embeddings, process in chronological chunks

### Challenge 2: Model Selection & Deployment
- **Issue**: Need to clarify what FinGPT components to use (model vs. implementation)
- **Solution**: 
  - Use llama3-sec (SEC-specific) as primary model via Ollama
  - Copy RAPTOR implementation from FinGPT's `rag.py`
  - Maintain flexibility to swap models (gpt-oss, qwen2.5, etc.)

### Challenge 3: Clustering Quality
- **Issue**: Poorly defined clusters reduce summary quality
- **Solution**: Use BIC for optimal cluster count, validate clusters manually on samples

### Challenge 4: Context Window Limits
- **Issue**: LLMs have token limits, can't ingest entire knowledge base
- **Solution**: RAPTOR's hierarchical retrieval provides most relevant chunks + summaries

### Challenge 5: Data Format Evolution
- **Issue**: SEC filing formats changed significantly between 1993 and 2024 (SGML → HTML → XML)
- **Solution**: Build robust parsing logic that handles multiple formats, test across time periods

### Challenge 6: Processing Time
- **Issue**: Unzipping and processing 51 GB of data could take significant time
- **Solution**: Parallel processing where possible, start with subset (one year) to validate pipeline

---

## Repository Structure
```
edgar_anomaly_detection/
├── data/
│   ├── external/         # Downloaded filings 1993-2024 (~51 GB, gitignored)
│   ├── processed/        # Extracted, chunked filings (gitignored)
│   └── embeddings/       # Generated embeddings (gitignored)
├── src/
│   ├── data/
│   │   ├── filing_extractor.py    # Extract full filing text
│   │   └── text_processor.py       # Chunk complete filings
│   ├── models/
│   │   ├── raptor.py              # RAPTOR class (adapted from FinGPT)
│   │   ├── embedding_generator.py
│   │   └── clustering.py
│   └── pipeline/
│       └── knowledge_base_builder.py
├── notebooks/
│   ├── 01_project_plan.ipynb      # This file
│   ├── 02_data_collection.ipynb
│   └── 03_raptor_testing.ipynb
├── dashboard/
│   └── README.md                  # Open WebUI setup instructions
├── .gitignore
├── requirements.txt
└── README.md
```

---

## Next Steps
1. ✅ Pull appropriate Ollama model (llama3-sec downloading)
2. [ ] Test llama3-sec model with sample SEC filing queries
3. [ ] Copy RAPTOR class from FinGPT GitHub to `src/models/raptor.py`
4. [ ] Extract sample filings from different time periods to examine format evolution
5. [ ] Build initial `filing_extractor.py` to handle HTML/XML/SGML parsing
6. [ ] Coordinate with team on EC2 instance access and GPU availability

---

## References
- **FinGPT GitHub:** https://github.com/AI4Finance-Foundation/FinGPT
- **FinGPT RAPTOR Implementation:** https://github.com/AI4Finance-Foundation/FinGPT/blob/master/fingpt/FinGPT_FinancialReportAnalysis/utils/rag.py
- **FinGPT Models on Hugging Face:** https://huggingface.co/AI4Finance-Foundation
- **RAPTOR RAG Documentation:** https://deepwiki.com/AI4Finance-Foundation/FinGPT/5.1-raptor-rag-system
- **SEC EDGAR API:** https://www.sec.gov/edgar/sec-api-documentation
- **Ollama:** https://ollama.ai/
- **Ollama Models Library:** https://ollama.com/library
- **Arcee AI llama3-sec:** https://ollama.com/arcee-ai/llama3-sec
- **Open WebUI:** https://github.com/open-webui/open-webui